In [ ]:
%cd ../out/

In [ ]:
import pandas as pd
import seaborn.objects as so

## Hyperparameters

### Quantile regression

In [ ]:
def get_hyperparams_quantile(fmt_str):
    dfs = []
    for alpha in [0.01, 0.05, 0.1, 0.2]:
        df = pd.read_csv(fmt_str.format(alpha=alpha))
        df['alpha'] = alpha
        dfs.append(df)
    df = pd.concat(dfs).reset_index(drop=True)

    best_lr, best_l2reg = df.groupby(['lr', 'l2reg'])['loss'].mean().idxmin()
    print(f'best lr: {best_lr:.3g}, best l2reg: {best_l2reg:.3g}')

    df['lr_str'] = df['lr'].map(lambda x: f'{x:.1g}')
    p = (
        so.Plot(df, x='lr_str', y='loss', color='l2reg')
        .add(so.Dots(), so.Dodge())
        .add(so.Dot(), so.Agg(), so.Dodge())
        .add(so.Range(), so.Est(errorbar='sd'), so.Dodge())
        .facet('alpha', wrap=2)
        .layout(size=(10, 10))
    )
    display(p)

In [ ]:
get_hyperparams_quantile('portfolio_yf_quantile/hyperparams_a{alpha:.2g}.csv')

In [ ]:
get_hyperparams_quantile('portfolio_yf_quantile_shuffled/hyperparams_a{alpha:.2g}.csv')

### Gaussian regression

In [ ]:
def get_hyperparams_gaussian(csv_path, ylim=(None, None)):
    df = pd.read_csv(csv_path)
    df['is_nan'] = df['loss'].isna().astype(int)
    with pd.option_context('display.float_format', '{:.4g}'.format):
        display(df.groupby(['lr', 'l2reg']).agg({'loss': ['mean', 'std'], 'is_nan': 'sum'}))

    best_lr, best_l2reg = df.groupby(['lr', 'l2reg'])['loss'].mean().idxmin()
    print(f'best lr: {best_lr:.3g}, best l2reg: {best_l2reg:.3g}')

    df['lr_str'] = df['lr'].map(lambda x: f'{x:.1g}')
    p = (
        so.Plot(df, x='lr_str', y='loss', color='l2reg')
        .add(so.Dots(), so.Dodge())
        .add(so.Dot(), so.Agg(), so.Dodge())
        .add(so.Range(), so.Est(errorbar='sd'), so.Dodge())
        .layout(size=(5, 5))
        .limit(y=ylim)
    )
    display(p)

In [ ]:
get_hyperparams_gaussian('portfolio_yf_gaussian/hyperparams.csv', ylim=(0, 100))

In [ ]:
get_hyperparams_gaussian('portfolio_yf_gaussian_shuffled/hyperparams.csv', ylim=(0, 100))

## ETO vs. E2E

In [ ]:
def plot_eto_vs_e2e(long_df, num_rows):
    return (
        so.Plot(long_df, x='alpha', y='value', color='model')
        .facet(row='metric', col='split')
        .add(so.Dots(), so.Dodge(), so.Jitter(.5))
        .share(y='row')
        .scale(x=so.Nominal())
        .layout(size=(10, num_rows * 3))
    )

def print_eto_e2e_results(df):
    best_e2e_models = (
        df[~df['val_task_loss'].isna()].groupby(['alpha', 'model'])['val_task_loss']
        .mean().unstack().idxmin(axis=1)
    )

    e2e_results_dict = {}
    for alpha in best_e2e_models.index:
        model = best_e2e_models.loc[alpha]
        mask = (df['alpha'] == alpha) & (df['model'] == model)
        new_df = df.loc[mask, ['test_task_loss', 'test_coverage']].agg(['mean', 'std'])
        e2e_results_dict[alpha] = new_df.unstack()
    best_e2e = pd.DataFrame(e2e_results_dict).T
    print('E2E results:')
    display(best_e2e)

    eto_results = (
        df.loc[df['model'] == 'eto']
        .groupby('alpha')[['test_task_loss', 'test_coverage']]
        .agg(['mean', 'std'])
    )
    print('ETO results:')
    display(eto_results)
    return

def get_df(eto_fmt_str, eto_cols, e2e_fmt_str, e2e_cols):
    dfs = []
    for alpha in [0.01, 0.05, 0.1, 0.2]:
        try:
            df_eto = pd.read_csv(eto_fmt_str.format(alpha=alpha))
            df_eto['model'] = 'eto'
            df_eto['alpha'] = alpha
            dfs.append(df_eto[eto_cols])
        except Exception as e:
            print(e)

    for alpha in [0.01, 0.05, 0.1, 0.2]:
        for lr in [0.0316, 1e-3, 1e-4, 1e-5]:
            try:
                df_e2e = pd.read_csv(e2e_fmt_str.format(alpha=alpha, lr=lr))
                df_e2e['model'] = f'e2e_lr{lr:.3g}'
                df_e2e['alpha'] = alpha
                dfs.append(df_e2e[e2e_cols])
            except FileNotFoundError as e:
                print(e)

    df = pd.concat(dfs)
    return df

def convert_to_long_df(df):
    long_df = df.melt(id_vars=['model', 'alpha', 'seed'], var_name='metric', value_name='value')
    long_df['split'] = long_df['metric'].map(lambda x: x.split('_')[0])
    long_df['metric'] = long_df['metric'].map(lambda x: x.split('_', maxsplit=1)[1])
    return long_df

### Quantile regression

In [ ]:
cols = [
    'model', 'alpha', 'seed',
    'train_pinball_loss', 'train_task_loss',
    'train_coverage', 'train_coverage_no_conformal',
    'test_pinball_loss', 'test_task_loss',
    'test_coverage', 'test_coverage_no_conformal'
]

In [ ]:
df = get_df(
    eto_fmt_str='portfolio_yf_quantile/eto_a{alpha:.2f}_lr0.0316_reg0.001.csv',
    eto_cols=cols,
    e2e_fmt_str='portfolio_yf_quantile/e2e_finetune_a{alpha:.2f}_lr{lr:.3g}_reg0.001.csv',
    e2e_cols=cols + ['val_task_loss']
)
print_eto_e2e_results(df)
long_df = convert_to_long_df(df.drop('val_task_loss', axis=1))
plot_eto_vs_e2e(long_df, num_rows=4)

In [ ]:
df = get_df(
    eto_fmt_str='portfolio_yf_quantile_shuffled/eto_a{alpha:.2f}_lr0.0316_reg0.0001.csv',
    eto_cols=cols,
    e2e_fmt_str='portfolio_yf_quantile_shuffled/e2e_finetune_a{alpha:.2f}_lr{lr:.3g}_reg0.0001.csv',
    e2e_cols=cols + ['val_task_loss']
)
print_eto_e2e_results(df)
long_df = convert_to_long_df(df.drop('val_task_loss', axis=1))
plot_eto_vs_e2e(long_df, num_rows=4)

### Gaussian regression

In [ ]:
cols = [
    'model', 'alpha', 'seed',
    'train_nll_loss', 'train_task_loss',
    'train_coverage', 'train_coverage_no_conformal',
    'test_nll_loss', 'test_task_loss',
    'test_coverage', 'test_coverage_no_conformal'
]

In [ ]:
# best lr: 0.001, best l2reg: 0.001

df = get_df(
    eto_fmt_str='portfolio_yf_gaussian/eto_a{alpha:.2f}_lr0.001_reg0.001.csv',
    eto_cols=cols,
    e2e_fmt_str='portfolio_yf_gaussian/e2e_finetune_a{alpha:.2f}_lr{lr:.3g}_reg0.001.csv',
    e2e_cols=cols + ['val_task_loss']
)
print_eto_e2e_results(df)
long_df = convert_to_long_df(df.drop('val_task_loss', axis=1))
plot_eto_vs_e2e(long_df, num_rows=4)

In [ ]:
df = get_df(
    eto_fmt_str='portfolio_yf_gaussian_shuffled/eto_a{alpha:.2f}_lr0.001_reg0.csv',
    eto_cols=cols,
    e2e_fmt_str='portfolio_yf_gaussian_shuffled/e2e_finetune_a{alpha:.2f}_lr{lr:.3g}_reg0.csv',
    e2e_cols=cols + ['val_task_loss']
)
print_eto_e2e_results(df)
long_df = convert_to_long_df(df.drop('val_task_loss', axis=1))
plot_eto_vs_e2e(long_df, num_rows=4)

### PICNN

In [ ]:
def get_picnn_eto_df(eto_fmt_str, eto_cols):
    dfs = []
    for alpha in [0.01, 0.05, 0.1, 0.2]:
        for lr in [1e-3, 1e-4]:
            for l2reg in [1e-3, 1e-4]:
                try:
                    df_eto = pd.read_csv(eto_fmt_str.format(alpha=alpha, lr=lr, l2reg=l2reg))
                    df_eto['lr'] = lr
                    df_eto['l2reg'] = l2reg
                    df_eto['model'] = f'eto_lr{lr:.3g}_reg{l2reg:.3g}'
                    df_eto['alpha'] = alpha
                    dfs.append(df_eto[eto_cols])
                except FileNotFoundError as e:
                    print(e)

    df = pd.concat(dfs)
    return df

def get_picnn_e2e_df(e2e_fmt_str, e2e_cols):
    dfs = []
    for alpha in [0.01, 0.05, 0.1, 0.2]:
        for lr in [1e-2, 1e-3, 1e-4]:
            for l2reg in [1e-3, 1e-4]:
                try:
                    df_eto = pd.read_csv(e2e_fmt_str.format(alpha=alpha, lr=lr, l2reg=l2reg))
                    df_eto['lr'] = lr
                    df_eto['l2reg'] = l2reg
                    df_eto['model'] = f'e2e_lr{lr:.3g}_reg{l2reg:.3g}'
                    df_eto['alpha'] = alpha
                    dfs.append(df_eto[e2e_cols])
                except FileNotFoundError as e:
                    print(e)

    df = pd.concat(dfs)
    return df

def print_picnn_results(df, by_col):
    results_dict = {}
    best_hyps = (
        df.groupby(['alpha', 'lr', 'l2reg'])[by_col]
        .mean()
        .unstack('alpha')
        .idxmin(axis=0)
    )
    for alpha in best_hyps.index:
        lr, l2reg = best_hyps.loc[alpha]
        mask = (df['alpha'] == alpha) & (df['lr'] == lr) & (df['l2reg'] == l2reg)
        new_df = df.loc[mask, ['test_task_loss', 'test_coverage']].agg(['mean', 'std'])
        results_dict[alpha] = new_df.unstack()
    best_results = pd.DataFrame(results_dict).T
    display(best_results)

In [ ]:
cols = [
    'model', 'alpha', 'seed', 'lr', 'l2reg',
    'train_task_loss', 'train_coverage',
    'test_task_loss', 'test_coverage',
]

In [ ]:
df = get_picnn_eto_df(
    eto_fmt_str='portfolio_yf_picnn_shuffle/eto_a{alpha:.2f}_L2_d64_lr{lr:.3g}_reg{l2reg:.3g}.csv',
    eto_cols=cols
)
print_picnn_results(df, by_col='train_task_loss')
long_df = convert_to_long_df(df.drop(['lr', 'l2reg'], axis=1))
plot_eto_vs_e2e(long_df, num_rows=2)

In [ ]:
display(df.groupby(['alpha', 'lr', 'l2reg']).agg({'train_task_loss': ['mean', 'std', 'count']}))

for alpha in [0.01, 0.05, 0.1, 0.2]:
    best_lr, best_l2reg = df[df['alpha'] == alpha].groupby(['lr', 'l2reg'])['train_task_loss'].mean().idxmin()
    print(f'alpha {alpha}: lr = {best_lr:.3g}, l2reg = {best_l2reg:.3g}')

In [ ]:
df = get_picnn_e2e_df(
    e2e_fmt_str='portfolio_yf_picnn_shuffle/e2e_finetune_a{alpha:.2f}_L2_d64_lr{lr:.3g}_reg{l2reg:.3g}.csv',
    e2e_cols=cols + ['val_task_loss']
)
print_picnn_results(df, by_col='val_task_loss')
long_df = convert_to_long_df(df.drop(['val_task_loss', 'lr', 'l2reg'], axis=1))
plot_eto_vs_e2e(long_df, num_rows=2)